In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import xgboost as xgb
import gc
from sklearn.feature_extraction.text import TfidfVectorizer
import re, string
from sklearn.metrics import log_loss




In [36]:

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "/home/king/git/Kaggle"]).decode("utf8"))

train = pd.read_csv('/home/king/git/Kaggle/final_train.csv')
test = pd.read_csv('/home/king/git/Kaggle/test.csv')

train = train.fillna("unknown")
test = test.fillna("unknown")


train_mes, valid_mes, train_l, valid_l = train_test_split(train['comment_text'],train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']], test_size=0.2, random_state=2)


Train Toxicity Model.ipynb
XGBOOST.ipynb
data
feature_engineering.ipynb
feature_importance.ipynb
final_train.csv
full-list-of-bad-words-banned-by-google-csv-file_2013_11_26_04_52_30_695.csv
model_tool.py
model_tool.pyc
models
nbsvm.ipynb
nbsvm_final_submission.csv
output.csv
sample_submission.csv
sample_submission.csv.zip
test.csv
test.csv.zip
test1.csv
train.csv
train.csv.zip
train_kaggle.csv



In [ ]:

'''
def text_process(comment):
    nopunc = [char for char in comment if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
'''
#Couldnt remove the stop words using the above function since it is taking too long
#Can try it on a local machine, I feel it improves the score-Not sure though


'''
transform_com = CountVectorizer().fit(pd.concat([train['comment_text'],test['comment_text']],axis=0))
comments_train = transform_com.transform(train['comment_text'])
comments_test = transform_com.transform(test['comment_text'])
gc.collect()'''


In [3]:
def tokenize(s): return re_tok.sub(r' \1 ', s).split()


In [4]:
#Using the tokenize function from Jeremy's kernel
re_tok = re.compile('([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
transform_com = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1).fit(pd.concat([train['comment_text'],test['comment_text']],axis=0))

In [33]:
'''comments_train = transform_com.transform(train['comment_text'])'''
comments_train = transform_com.transform(train['comment_text'])
comments_test = transform_com.transform(test['comment_text'])
gc.collect()


def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=2017, num_rounds=400):
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.12
    param['max_depth'] = 5
    param['silent'] = 1
    param['eval_metric'] = 'logloss'
    param['min_child_weight'] = 1
    param['subsample'] = 0.5
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return model
    


In [17]:
from scipy.sparse import hstack

In [34]:

col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((test.shape[0], len(col)))
fold_count = 10
fold_size = len(train['comment_text']) // fold_count
models = []
X = train['comment_text']
y = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]
test_predicts_list = []
final_loss = []
for fold_id in range(0, fold_count):
    fold_start = fold_size * fold_id
    fold_end = fold_start + fold_size
            
    if fold_id == fold_size - 1:
        fold_end = len(X)

    train_x = np.concatenate([X[:fold_start], X[fold_end:]])
    #train_x = hstack([X[:fold_start], X[fold_end:]])

    train_y = np.concatenate([y[:fold_start], y[fold_end:]])

    val_x = X[fold_start:fold_end]
    val_y = y[fold_start:fold_end] 
    train_x = transform_com.transform(train_x)
    val_x = transform_com.transform(val_x)
    
    loss = []
    for i, j in enumerate(col):
        print('fit '+j)
        model = runXGB(train_x, train_y[j], val_x,val_y[j])
        #loss.append(log_loss(val_y[j],pred_test_y))
        preds[:,i] = model.predict_proba(xgb.DMatrix(comments_test))
        gc.collect()
    #total_loss = np.mean(loss)
    #final_loss.append(total_loss)
    test_predicts_list.append(preds)
test_predicts = np.ones(test_predicts_list[0].shape)
for fold_predict in test_predicts_list:
    test_predicts *= fold_predict

test_predicts **= (1. / len(test_predicts_list))
#print("final loss {}".format(np.mean(final_loss)))

fit toxic


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
subm = pd.read_csv('/home/king/git/Kaggle/sample_submission.csv')    
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(test_predicts, columns = col)], axis=1)
submission.to_csv('xgb.csv', index=False)

In [37]:
comments_train = transform_com.transform(train_mes)
comments_valid = transform_com.transform(valid_mes)
comments_test = transform_com.transform(test['comment_text'])
gc.collect()



import xgboost as xgb
'''
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=2, num_rounds=400):
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.1
    param['max_depth'] = 5
    param['silent'] = 1
    param['eval_metric'] = 'logloss'
    param['min_child_weight'] = 1
    param['subsample'] = 0.4
    param['colsample_bytree'] = 0.6
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)
    xgtest = xgb.DMatrix(test_X)
    model = xgb.train(plst, xgtrain, num_rounds)
    pred_test_y = model.predict(xgtest)
    return pred_test_y, model
'''


def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=2017, num_rounds=400):
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.12
    param['max_depth'] = 5
    param['silent'] = 1
    param['eval_metric'] = 'logloss'
    param['min_child_weight'] = 1
    param['subsample'] = 0.5
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return model
    

col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((test.shape[0], len(col)))

for i, j in enumerate(col):
    print('fit '+j)
    model = runXGB(comments_train, train_l[j], comments_valid,valid_l[j]
    name = "pima.pickle" + str(i) + ".dat"
    pickle.dump(model, open(name, "wb"))
    preds[:,i] = model.predict(xgb.DMatrix(comments_test))
    gc.collect()



fit toxic
[0]	train-logloss:0.612846	test-logloss:0.612952
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 20 rounds.
[1]	train-logloss:0.548545	test-logloss:0.548681
[2]	train-logloss:0.496794	test-logloss:0.496986
[3]	train-logloss:0.455542	test-logloss:0.455798
[4]	train-logloss:0.419936	test-logloss:0.420156
[5]	train-logloss:0.390663	test-logloss:0.390941
[6]	train-logloss:0.365906	test-logloss:0.366276
[7]	train-logloss:0.34537	test-logloss:0.345747
[8]	train-logloss:0.327801	test-logloss:0.32816
[9]	train-logloss:0.312753	test-logloss:0.313235
[10]	train-logloss:0.299802	test-logloss:0.300325
[11]	train-logloss:0.289397	test-logloss:0.290014
[12]	train-logloss:0.279648	test-logloss:0.280395
[13]	train-logloss:0.271687	test-logloss:0.272492
[14]	train-logloss:0.264024	test-logloss:0.264853
[15]	train-logloss:0.257649	test-logloss:0.258559
[16]	train-logloss:0.252226	test-logloss:0.253231
[17

[161]	train-logloss:0.15233	test-logloss:0.161749
[162]	train-logloss:0.152106	test-logloss:0.161594
[163]	train-logloss:0.151884	test-logloss:0.161412
[164]	train-logloss:0.151692	test-logloss:0.161295
[165]	train-logloss:0.15152	test-logloss:0.1612
[166]	train-logloss:0.151376	test-logloss:0.161115
[167]	train-logloss:0.151201	test-logloss:0.160997
[168]	train-logloss:0.151038	test-logloss:0.160907
[169]	train-logloss:0.150833	test-logloss:0.160726
[170]	train-logloss:0.150705	test-logloss:0.160654
[171]	train-logloss:0.150489	test-logloss:0.160476
[172]	train-logloss:0.150159	test-logloss:0.160178
[173]	train-logloss:0.149997	test-logloss:0.160064
[174]	train-logloss:0.149822	test-logloss:0.159924
[175]	train-logloss:0.149534	test-logloss:0.159707
[176]	train-logloss:0.149324	test-logloss:0.159523
[177]	train-logloss:0.149185	test-logloss:0.159416
[178]	train-logloss:0.14902	test-logloss:0.159302
[179]	train-logloss:0.148855	test-logloss:0.159185
[180]	train-logloss:0.14865	test-log

[323]	train-logloss:0.131103	test-logloss:0.146329
[324]	train-logloss:0.131021	test-logloss:0.146261
[325]	train-logloss:0.130951	test-logloss:0.146206
[326]	train-logloss:0.130847	test-logloss:0.146169
[327]	train-logloss:0.130625	test-logloss:0.146021
[328]	train-logloss:0.130484	test-logloss:0.145945
[329]	train-logloss:0.130413	test-logloss:0.145908
[330]	train-logloss:0.130358	test-logloss:0.145871
[331]	train-logloss:0.130231	test-logloss:0.145796
[332]	train-logloss:0.13011	test-logloss:0.145701
[333]	train-logloss:0.130007	test-logloss:0.145647
[334]	train-logloss:0.12989	test-logloss:0.145557
[335]	train-logloss:0.129769	test-logloss:0.14548
[336]	train-logloss:0.129674	test-logloss:0.145399
[337]	train-logloss:0.129523	test-logloss:0.145283
[338]	train-logloss:0.12935	test-logloss:0.145162
[339]	train-logloss:0.129223	test-logloss:0.145072
[340]	train-logloss:0.129123	test-logloss:0.145004
[341]	train-logloss:0.128997	test-logloss:0.144927
[342]	train-logloss:0.128932	test-l

[83]	train-logloss:0.021454	test-logloss:0.025497
[84]	train-logloss:0.021359	test-logloss:0.025445
[85]	train-logloss:0.0213	test-logloss:0.0254
[86]	train-logloss:0.021237	test-logloss:0.025357
[87]	train-logloss:0.021181	test-logloss:0.025343
[88]	train-logloss:0.021125	test-logloss:0.025293
[89]	train-logloss:0.021072	test-logloss:0.025256
[90]	train-logloss:0.021008	test-logloss:0.025238
[91]	train-logloss:0.020935	test-logloss:0.025187
[92]	train-logloss:0.020867	test-logloss:0.02514
[93]	train-logloss:0.020767	test-logloss:0.025065
[94]	train-logloss:0.020721	test-logloss:0.025037
[95]	train-logloss:0.020643	test-logloss:0.024997
[96]	train-logloss:0.020584	test-logloss:0.02496
[97]	train-logloss:0.020533	test-logloss:0.024914
[98]	train-logloss:0.020482	test-logloss:0.02484
[99]	train-logloss:0.020421	test-logloss:0.024797
[100]	train-logloss:0.020364	test-logloss:0.024763
[101]	train-logloss:0.020276	test-logloss:0.024732
[102]	train-logloss:0.020191	test-logloss:0.024685
[103

[245]	train-logloss:0.01477	test-logloss:0.021738
[246]	train-logloss:0.014749	test-logloss:0.021732
[247]	train-logloss:0.014723	test-logloss:0.021716
[248]	train-logloss:0.014686	test-logloss:0.021709
[249]	train-logloss:0.014667	test-logloss:0.021693
[250]	train-logloss:0.014639	test-logloss:0.02166
[251]	train-logloss:0.014611	test-logloss:0.021641
[252]	train-logloss:0.014598	test-logloss:0.021631
[253]	train-logloss:0.014569	test-logloss:0.021628
[254]	train-logloss:0.014539	test-logloss:0.02161
[255]	train-logloss:0.014501	test-logloss:0.021591
[256]	train-logloss:0.014472	test-logloss:0.02157
[257]	train-logloss:0.014444	test-logloss:0.021557
[258]	train-logloss:0.014413	test-logloss:0.021539
[259]	train-logloss:0.014394	test-logloss:0.021526
[260]	train-logloss:0.014362	test-logloss:0.021504
[261]	train-logloss:0.01433	test-logloss:0.021475
[262]	train-logloss:0.014296	test-logloss:0.021459
[263]	train-logloss:0.014276	test-logloss:0.021452
[264]	train-logloss:0.014253	test-lo

...

[12]	train-logloss:0.200129	test-logloss:0.201994
[13]	train-logloss:0.190303	test-logloss:0.192285
[14]	train-logloss:0.181451	test-logloss:0.183549
[15]	train-logloss:0.173761	test-logloss:0.176005
[16]	train-logloss:0.167192	test-logloss:0.169503
[17]	train-logloss:0.16131	test-logloss:0.163717
[18]	train-logloss:0.156318	test-logloss:0.158923
[19]	train-logloss:0.151916	test-logloss:0.154628
[20]	train-logloss:0.147683	test-logloss:0.150496
[21]	train-logloss:0.144084	test-logloss:0.146984
[22]	train-logloss:0.140959	test-logloss:0.14396
[23]	train-logloss:0.138184	test-logloss:0.141309
[24]	train-logloss:0.13582	test-logloss:0.139036
[25]	train-logloss:0.133647	test-logloss:0.136918
[26]	train-logloss:0.131546	test-logloss:0.134862
[27]	train-logloss:0.129563	test-logloss:0.133058
[28]	train-logloss:0.127676	test-logloss:0.131206
[29]	train-logloss:0.126095	test-logloss:0.129656
[30]	train-logloss:0.12443	test-logloss:0.128089
[31]	train-logloss:0.122983	test-logloss:0.126784
[32]

[176]	train-logloss:0.079394	test-logloss:0.09016
[177]	train-logloss:0.079289	test-logloss:0.090078
[178]	train-logloss:0.079185	test-logloss:0.090003
[179]	train-logloss:0.079002	test-logloss:0.08987
[180]	train-logloss:0.078945	test-logloss:0.089825
[181]	train-logloss:0.078857	test-logloss:0.089784
[182]	train-logloss:0.078795	test-logloss:0.08975
[183]	train-logloss:0.078729	test-logloss:0.089714
[184]	train-logloss:0.078635	test-logloss:0.089654
[185]	train-logloss:0.078515	test-logloss:0.089567
[186]	train-logloss:0.078429	test-logloss:0.08953
[187]	train-logloss:0.078264	test-logloss:0.089406
[188]	train-logloss:0.07817	test-logloss:0.089361
[189]	train-logloss:0.078113	test-logloss:0.089302
[190]	train-logloss:0.078004	test-logloss:0.089236
[191]	train-logloss:0.07794	test-logloss:0.089212
[192]	train-logloss:0.07787	test-logloss:0.089163
[193]	train-logloss:0.077733	test-logloss:0.089088
[194]	train-logloss:0.077654	test-logloss:0.089026
[195]	train-logloss:0.077474	test-logl

[338]	train-logloss:0.066578	test-logloss:0.08174
[339]	train-logloss:0.0665	test-logloss:0.0817
[340]	train-logloss:0.066445	test-logloss:0.081653
[341]	train-logloss:0.066369	test-logloss:0.081613
[342]	train-logloss:0.066335	test-logloss:0.081593
[343]	train-logloss:0.066268	test-logloss:0.081563
[344]	train-logloss:0.066203	test-logloss:0.081534
[345]	train-logloss:0.066163	test-logloss:0.081508
[346]	train-logloss:0.066127	test-logloss:0.081487
[347]	train-logloss:0.066093	test-logloss:0.081469
[348]	train-logloss:0.066005	test-logloss:0.08141
[349]	train-logloss:0.065943	test-logloss:0.08137
[350]	train-logloss:0.065889	test-logloss:0.081338
[351]	train-logloss:0.065861	test-logloss:0.081331
[352]	train-logloss:0.065784	test-logloss:0.081277
[353]	train-logloss:0.06573	test-logloss:0.081243
[354]	train-logloss:0.065679	test-logloss:0.081212
[355]	train-logloss:0.065609	test-logloss:0.081193
[356]	train-logloss:0.065522	test-logloss:0.081104
[357]	train-logloss:0.065474	test-loglo

[99]	train-logloss:0.0205	test-logloss:0.024364
[100]	train-logloss:0.020453	test-logloss:0.024328
[101]	train-logloss:0.020351	test-logloss:0.024263
[102]	train-logloss:0.020276	test-logloss:0.024227
[103]	train-logloss:0.020178	test-logloss:0.024145
[104]	train-logloss:0.020064	test-logloss:0.024073
[105]	train-logloss:0.019992	test-logloss:0.024013
[106]	train-logloss:0.019928	test-logloss:0.023955
[107]	train-logloss:0.019874	test-logloss:0.02392
[108]	train-logloss:0.019802	test-logloss:0.023872
[109]	train-logloss:0.019734	test-logloss:0.023823
[110]	train-logloss:0.01968	test-logloss:0.0238
[111]	train-logloss:0.01963	test-logloss:0.023775
[112]	train-logloss:0.019563	test-logloss:0.023744
[113]	train-logloss:0.019511	test-logloss:0.023702
[114]	train-logloss:0.019426	test-logloss:0.023651
[115]	train-logloss:0.019324	test-logloss:0.023601
[116]	train-logloss:0.019177	test-logloss:0.023505
[117]	train-logloss:0.019126	test-logloss:0.023465
[118]	train-logloss:0.019059	test-loglo

[261]	train-logloss:0.013007	test-logloss:0.019359
[262]	train-logloss:0.012968	test-logloss:0.019328
[263]	train-logloss:0.012925	test-logloss:0.019295
[264]	train-logloss:0.012903	test-logloss:0.019285
[265]	train-logloss:0.012885	test-logloss:0.019265
[266]	train-logloss:0.012856	test-logloss:0.019243
[267]	train-logloss:0.012809	test-logloss:0.019214
[268]	train-logloss:0.012789	test-logloss:0.019197
[269]	train-logloss:0.012771	test-logloss:0.019178
[270]	train-logloss:0.01275	test-logloss:0.01916
[271]	train-logloss:0.01272	test-logloss:0.019148
[272]	train-logloss:0.012691	test-logloss:0.01913
[273]	train-logloss:0.012666	test-logloss:0.019116
[274]	train-logloss:0.012656	test-logloss:0.019106
[275]	train-logloss:0.012634	test-logloss:0.019104
[276]	train-logloss:0.012601	test-logloss:0.019076
[277]	train-logloss:0.012585	test-logloss:0.01907
[278]	train-logloss:0.012546	test-logloss:0.019068
[279]	train-logloss:0.01253	test-logloss:0.019053
[280]	train-logloss:0.012503	test-log

In [39]:
subm = pd.read_csv('/home/king/git/Kaggle/sample_submission.csv')    
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = col)], axis=1)
submission.to_csv('xgb.csv', index=False)